# Create Networks - Advanced

This tutorial shows how to create a more complex pandapower network step by step. The network includes every element which is availiable in the pandapower framework.

The final network looks like this:

<img src="pics/example_network.png" width="50%">

The structural information about this network are stored in csv tables in the example_advanced folder.

For a better overview the creation of the individual components is divided in three steps. Each step handles one of the three voltage levels: high, medium and low voltage. We star by initializing an empty pandapower network:

In [ ]:
import pandas as pd

from pandapower.create import (
    create_empty_network,
    create_ext_grid,
    create_bus,
    create_line,
    create_transformer_from_parameters,
    create_transformer3w_from_parameters,
    create_switch,
    create_gen,
    create_load,
    create_sgen,
    create_shunt,
    create_xward,
    create_impedance
)
from pandapower.run import runpp
from pandapower.toolbox.element_selection import get_element_index
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [1]:
#create an empty network
net = create_empty_network()

## High voltage level

### Buses

<img src="pics/example_network_buses_hv_detail.png" width="50%">


There are two 380 kV and five 110 kV busbars (type="b"). The 380/110 kV substation is modeled in detail with all nodes and switches, which is why we need additional nodes (type="b") to connect the switches.

In [2]:
# Double busbar
create_bus(net, name='Double Busbar 1', vn_kv=380, type='b')
create_bus(net, name='Double Busbar 2', vn_kv=380, type='b')
for i in range(10):
    create_bus(net, name='Bus DB T%s' % i, vn_kv=380, type='n')
for i in range(1, 5):
    create_bus(net, name='Bus DB %s' % i, vn_kv=380, type='n')

# Single busbar
create_bus(net, name='Single Busbar', vn_kv=110, type='b')
for i in range(1, 6):
    create_bus(net, name='Bus SB %s' % i, vn_kv=110, type='n')
for i in range(1, 6):
    for j in [1, 2]:
        create_bus(net, name='Bus SB T%s.%s' % (i, j), vn_kv=110, type='n')

# Remaining buses
for i in range(1, 5):
    create_bus(net, name=f'Bus HV{i}', vn_kv=110, type='n')

bus1 = get_element_index(net, "bus", "Bus HV1")


# show bustable
net.bus

,name,vn_kv,type,zone,in_service,geo
0,Double Busbar 1,380.0,b,None,True,None
1,Double Busbar 2,380.0,b,None,True,None
2,Bus DB T0,380.0,n,None,True,None
3,Bus DB T1,380.0,n,None,True,None
4,Bus DB T2,380.0,n,None,True,None
5,Bus DB T3,380.0,n,None,True,None
6,Bus DB T4,380.0,n,None,True,None
7,Bus DB T5,380.0,n,None,True,None
8,Bus DB T6,380.0,n,None,True,None
9,Bus DB T7,380.0,n,None,True,None


### Lines

<img src="pics/example_network_lines_hv.png" width="40%">

The information about the 6 HV lines are stored in a csv file that we load from the hard drive:

In [3]:
hv_lines = pd.read_csv('example_advanced/hv_lines.csv', sep=';', header=0, decimal=',')
hv_lines

,line_name,from_bus,to_bus,std_type,length,parallel
0,HV Line1,Bus SB 2,Bus HV1,184-AL1/30-ST1A 110.0,30,1
1,HV Line2,Bus HV1,Bus HV2,184-AL1/30-ST1A 110.0,20,1
2,HV Line3,Bus HV2,Bus HV4,184-AL1/30-ST1A 110.0,30,1
3,HV Line4,Bus HV1,Bus HV4,184-AL1/30-ST1A 110.0,15,1
4,HV Line5,Bus HV3,Bus HV4,184-AL1/30-ST1A 110.0,25,1
5,HV Line6,Bus SB 3,Bus HV3,184-AL1/30-ST1A 110.0,30,2


and use to create all lines:

In [4]:
# create lines
for _, hv_line in hv_lines.iterrows():
        from_bus = get_element_index(net, "bus", hv_line.from_bus)
        to_bus = get_element_index(net, "bus", hv_line.to_bus)
        create_line(net, from_bus, to_bus, length_km=hv_line.length,std_type=hv_line.std_type, name=hv_line.line_name, parallel=hv_line.parallel)

# show line table
net.line

,name,std_type,from_bus,to_bus,length_km,r_ohm_per_km,x_ohm_per_km,c_nf_per_km,g_us_per_km,max_i_ka,df,parallel,type,in_service,geo
0,HV Line1,184-AL1/30-ST1A 110.0,18,32,30.0,0.1571,0.4,8.8,0.0,0.535,1.0,1,ol,True,None
1,HV Line2,184-AL1/30-ST1A 110.0,32,33,20.0,0.1571,0.4,8.8,0.0,0.535,1.0,1,ol,True,None
2,HV Line3,184-AL1/30-ST1A 110.0,33,35,30.0,0.1571,0.4,8.8,0.0,0.535,1.0,1,ol,True,None
3,HV Line4,184-AL1/30-ST1A 110.0,32,35,15.0,0.1571,0.4,8.8,0.0,0.535,1.0,1,ol,True,None
4,HV Line5,184-AL1/30-ST1A 110.0,34,35,25.0,0.1571,0.4,8.8,0.0,0.535,1.0,1,ol,True,None
5,HV Line6,184-AL1/30-ST1A 110.0,19,34,30.0,0.1571,0.4,8.8,0.0,0.535,1.0,2,ol,True,None


### Transformer

<img src="pics/example_network_trafos_hv.png" width="40%">

The 380/110 kV transformer connects the buses "Bus DB 1" and "Bus DB 2". We use the get_element_index function from the pandapower toolbox to find the bus indices of the buses with these names and create a transformer by directly specifying the parameters:

In [5]:
hv_bus = get_element_index(net, "bus", "Bus DB 2")
lv_bus = get_element_index(net, "bus", "Bus SB 1")
create_transformer_from_parameters(net, hv_bus, lv_bus, sn_mva=300, vn_hv_kv=380, vn_lv_kv=110, vkr_percent=0.06,
                                      vk_percent=8, pfe_kw=0, i0_percent=0, tp_pos=0, shift_degree=0, name='EHV-HV-Trafo')

net.trafo # show trafo table

,name,std_type,hv_bus,lv_bus,sn_mva,vn_hv_kv,vn_lv_kv,vk_percent,vkr_percent,pfe_kw,...,tap_step_degree,tap_pos,tap_changer_type,id_characteristic_table,tap_dependency_table,parallel,df,in_service,tp_pos,oltc
0,EHV-HV-Trafo,None,13,17,300.0,380.0,110.0,8.0,0.06,0.0,...,NaN,NaN,NaN,<NA>,False,1,1.0,True,0.0,False


### Switches

<img src="pics/example_network_switches_hv.png" width="60%">

Now we create the switches to connect the buses in the transformer station. The switch configuration is stored in the following csv table:

In [6]:
hv_bus_sw = pd.read_csv('example_advanced/hv_bus_sw.csv', sep=';', header=0, decimal=',')
hv_bus_sw

,bus_name,from_bus,to_bus,type,et,closed
0,DB DS0,Double Busbar 2,Bus DB T0,DS,b,True
1,DB DS1,Double Busbar 1,Bus DB T1,DS,b,True
2,DB DS2,Double Busbar 2,Bus DB T3,DS,b,True
3,DB DS3,Double Busbar 1,Bus DB T3,DS,b,False
4,DB DS4,Double Busbar 2,Bus DB T5,DS,b,True
5,DB DS5,Double Busbar 1,Bus DB T5,DS,b,False
6,DB DS6,Double Busbar 2,Bus DB T7,DS,b,True
7,DB DS7,Double Busbar 1,Bus DB T7,DS,b,False
8,DB DS8,Double Busbar 2,Bus DB T9,DS,b,True
9,DB DS9,Double Busbar 1,Bus DB T9,DS,b,False


In [7]:
# Bus-bus switches
for _, switch in hv_bus_sw.iterrows():
    from_bus = get_element_index(net, "bus", switch.from_bus)
    to_bus = get_element_index(net, "bus", switch.to_bus)
    create_switch(net, from_bus, to_bus, et=switch.et, closed=switch.closed, type=switch.type, name=switch.bus_name)

# Bus-line switches
hv_buses = net.bus[(net.bus.vn_kv == 380) | (net.bus.vn_kv == 110)].index
hv_ls = net.line[(net.line.from_bus.isin(hv_buses)) & (net.line.to_bus.isin(hv_buses))]
for _, line in hv_ls.iterrows():
        create_switch(net, line.from_bus, line.name, et='l', closed=True, type='LBS', name=f"Switch {net.bus.name.at[line.from_bus]} - {line['name']}")
        create_switch(net, line.to_bus, line.name, et='l', closed=True, type='LBS', name=f"Switch {net.bus.name.at[line.to_bus]} - {line['name']}")

# Trafo-line switches
create_switch(net, get_element_index(net, "bus", 'Bus DB 2'), get_element_index(net, "trafo", 'EHV-HV-Trafo'), et='t', closed=True, type='LBS', name='Switch DB2 - EHV-HV-Trafo')
create_switch(net, get_element_index(net, "bus", 'Bus SB 1'), get_element_index(net, "trafo", 'EHV-HV-Trafo'), et='t', closed=True, type='LBS', name='Switch SB1 - EHV-HV-Trafo')

# show switch table
net.switch

,bus,element,et,type,closed,name,z_ohm,in_ka
0,1,2,b,DS,True,DB DS0,0.0,NaN
1,0,3,b,DS,True,DB DS1,0.0,NaN
2,1,5,b,DS,True,DB DS2,0.0,NaN
3,0,5,b,DS,False,DB DS3,0.0,NaN
4,1,7,b,DS,True,DB DS4,0.0,NaN
5,0,7,b,DS,False,DB DS5,0.0,NaN
6,1,9,b,DS,True,DB DS6,0.0,NaN
7,0,9,b,DS,False,DB DS7,0.0,NaN
8,1,11,b,DS,True,DB DS8,0.0,NaN
9,0,11,b,DS,False,DB DS9,0.0,NaN


### External Grid

<img src="pics/example_network_ext_grids_hv.png" width="40%">

We equip the high voltage side of the transformer with an external grid connection:

In [8]:
create_ext_grid(net, get_element_index(net, "bus", 'Double Busbar 1'), vm_pu=1.03, va_degree=0, name='External grid',
                   s_sc_max_mva=10000, rx_max=0.1, rx_min=0.1)

net.ext_grid # show external grid table

,name,bus,vm_pu,va_degree,slack_weight,in_service,controllable,s_sc_max_mva,rx_min,rx_max
0,External grid,0,1.03,0.0,1.0,True,False,10000.0,0.1,0.1


### Loads

<img src="pics/example_network_loads_hv.png" width="40%">

The five loads in the HV network are defined in the following csv file:

In [9]:
hv_loads = pd.read_csv('example_advanced/hv_loads.csv', sep=';', header=0, decimal=',')
hv_loads

,load_name,bus,p,q
0,MV Net 0,Bus SB 4,38,6
1,MV Net 1,Bus HV1,38,6
2,MV Net 2,Bus HV2,38,6
3,MV Net 3,Bus HV3,38,6
4,MV Net 4,Bus HV4,38,6


In [10]:
for _, load in hv_loads.iterrows():
    bus_idx = get_element_index(net, "bus", load.bus)
    create_load(net, bus_idx, p_mw=load.p, q_mvar=load.q, name=load.load_name)

# show load table
net.load

,name,bus,p_mw,q_mvar,const_z_p_percent,const_i_p_percent,const_z_q_percent,const_i_q_percent,sn_mva,scaling,in_service,type
0,MV Net 0,20,38.0,6.0,0.0,0.0,0.0,0.0,NaN,1.0,True,wye
1,MV Net 1,32,38.0,6.0,0.0,0.0,0.0,0.0,NaN,1.0,True,wye
2,MV Net 2,33,38.0,6.0,0.0,0.0,0.0,0.0,NaN,1.0,True,wye
3,MV Net 3,34,38.0,6.0,0.0,0.0,0.0,0.0,NaN,1.0,True,wye
4,MV Net 4,35,38.0,6.0,0.0,0.0,0.0,0.0,NaN,1.0,True,wye


### Generator

<img src="pics/example_network_gens_hv.png" width="40%">

The voltage controlled generator is created with an active power of 100 MW (negative for generation) and a voltage set point of 1.03 per unit:

In [11]:
create_gen(net, get_element_index(net, "bus", 'Bus HV4'), vm_pu=1.03, p_mw=100, name='Gas turbine')

# show generator table
net.gen

,name,bus,p_mw,vm_pu,sn_mva,min_q_mvar,max_q_mvar,scaling,slack,controllable,id_q_capability_characteristic,reactive_capability_curve,curve_style,in_service,slack_weight,type
0,Gas turbine,35,100.0,1.03,NaN,NaN,NaN,1.0,False,True,<NA>,False,NaN,True,0.0,None


### Static generators

<img src="pics/example_network_sgens_hv.png" width="40%">

We create this wind park with an active power of 20 MW (negative for generation) and a reactive power of -4 Mvar. To classify the generation as a wind park, we set type to "WP":

In [12]:
create_sgen(net, get_element_index(net, "bus", 'Bus SB 5'), p_mw=20, q_mvar=4, sn_mva=45, 
               type='WP', name='Wind Park')

# show static generator table
net.sgen

,name,bus,p_mw,q_mvar,min_q_mvar,max_q_mvar,sn_mva,scaling,controllable,id_q_capability_characteristic,reactive_capability_curve,curve_style,in_service,type,current_source
0,Wind Park,21,20.0,4.0,NaN,NaN,45.0,1.0,False,<NA>,False,NaN,True,WP,True


### Shunt

<img src="pics/example_network_shunts_hv.png" width="40%">

In [13]:
create_shunt(net, bus1, p_mw=0, q_mvar=0.960, name='Shunt')

# show shunt table
net.shunt

,bus,name,q_mvar,p_mw,vn_kv,step,max_step,id_characteristic_table,step_dependency_table,in_service
0,32,Shunt,0.96,0.0,110.0,1,1,<NA>,False,True


### External network equivalents


<img src="pics/example_network_ext_equi_hv.png" width="40%">

The two remaining elements are impedances and extended ward equivalents:

In [14]:
# Impedance
create_impedance(net, get_element_index(net, "bus", 'Bus HV3'), bus1,
                    rft_pu=0.074873, xft_pu=0.198872, sn_mva=100, name='Impedance')

# show impedance table
net.impedance

,name,from_bus,to_bus,rft_pu,xft_pu,rtf_pu,xtf_pu,gf_pu,bf_pu,gt_pu,bt_pu,sn_mva,in_service
0,Impedance,34,32,0.074873,0.198872,0.074873,0.198872,0.0,0.0,0.0,0.0,100.0,True


In [15]:
# xwards
create_xward(net, get_element_index(net, "bus", 'Bus HV3'), ps_mw=23.942, qs_mvar=-12.24187, pz_mw=2.814571, 
                qz_mvar=0, r_ohm=0, x_ohm=12.18951, vm_pu=1.02616, name='XWard 1')
create_xward(net, bus1, ps_mw=3.776, qs_mvar=-7.769979, pz_mw=9.174917,
                qz_mvar=0, r_ohm=0, x_ohm=50.56217, vm_pu=1.024001, name='XWard 2')

# show xward table
net.xward

,name,bus,ps_mw,qs_mvar,qz_mvar,pz_mw,r_ohm,x_ohm,vm_pu,slack_weight,in_service
0,XWard 1,34,23.942,-12.241870,0.0,2.814571,0.0,12.18951,1.026160,0.0,True
1,XWard 2,32,3.776,-7.769979,0.0,9.174917,0.0,50.56217,1.024001,0.0,True


## Medium voltage level

### Buses

<img src="pics/example_network_buses_mv.png" width="50%">

In [16]:
create_bus(net, name='Bus MV0 20kV', vn_kv=20, type='n')
for i in range(8):
    create_bus(net, name='Bus MV%s' % i, vn_kv=10, type='n')

#show only medium voltage bus table
mv_buses = net.bus[(net.bus.vn_kv == 10) | (net.bus.vn_kv == 20)]
mv_buses

,name,vn_kv,type,zone,in_service,geo
36,Bus MV0 20kV,20.0,n,None,True,None
37,Bus MV0,10.0,n,None,True,None
38,Bus MV1,10.0,n,None,True,None
39,Bus MV2,10.0,n,None,True,None
40,Bus MV3,10.0,n,None,True,None
41,Bus MV4,10.0,n,None,True,None
42,Bus MV5,10.0,n,None,True,None
43,Bus MV6,10.0,n,None,True,None
44,Bus MV7,10.0,n,None,True,None


### Lines

<img src="pics/example_network_lines_mv.png" width="50%">

In [17]:
mv_lines = pd.read_csv('example_advanced/mv_lines.csv', sep=';', header=0, decimal=',')
for _, mv_line in mv_lines.iterrows():
    from_bus = get_element_index(net, "bus", mv_line.from_bus)
    to_bus = get_element_index(net, "bus", mv_line.to_bus)
    create_line(net, from_bus, to_bus, length_km=mv_line.length, std_type=mv_line.std_type, name=mv_line.line_name)

# show only medium voltage lines
net.line[net.line.from_bus.isin(mv_buses.index)]

,name,std_type,from_bus,to_bus,length_km,r_ohm_per_km,x_ohm_per_km,c_nf_per_km,g_us_per_km,max_i_ka,df,parallel,type,in_service,geo
6,MV Line1,NA2XS2Y 1x185 RM/25 12/20 kV,37,38,1.5,0.161,0.117,273.0,0.0,0.362,1.0,1,cs,True,None
7,MV Line2,NA2XS2Y 1x185 RM/25 12/20 kV,38,39,1.5,0.161,0.117,273.0,0.0,0.362,1.0,1,cs,True,None
8,MV Line3,NA2XS2Y 1x185 RM/25 12/20 kV,39,40,1.5,0.161,0.117,273.0,0.0,0.362,1.0,1,cs,True,None
9,MV Line4,NA2XS2Y 1x185 RM/25 12/20 kV,40,41,1.5,0.161,0.117,273.0,0.0,0.362,1.0,1,cs,True,None
10,MV Line5,NA2XS2Y 1x185 RM/25 12/20 kV,41,42,1.5,0.161,0.117,273.0,0.0,0.362,1.0,1,cs,True,None
11,MV Line6,NA2XS2Y 1x185 RM/25 12/20 kV,42,43,1.5,0.161,0.117,273.0,0.0,0.362,1.0,1,cs,True,None
12,MV Line7,NA2XS2Y 1x185 RM/25 12/20 kV,43,44,1.5,0.161,0.117,273.0,0.0,0.362,1.0,1,cs,True,None
13,MV Line8,NA2XS2Y 1x185 RM/25 12/20 kV,37,44,1.5,0.161,0.117,273.0,0.0,0.362,1.0,1,cs,True,None


### 3 Winding Transformer

<img src="pics/example_network_trafos_mv.png" width="50%">

The three winding transformer transforms its high voltage level to two different lower voltage levels, in this case from 110 kV to 20 kV and 10 kV.

In [18]:
hv_bus = get_element_index(net, "bus", "Bus HV2")
mv_bus = get_element_index(net, "bus", "Bus MV0 20kV")
lv_bus = get_element_index(net, "bus", "Bus MV0")
create_transformer3w_from_parameters(net, hv_bus, mv_bus, lv_bus, vn_hv_kv=110, vn_mv_kv=20, vn_lv_kv=10, 
                                        sn_hv_mva=40, sn_mv_mva=15, sn_lv_mva=25, vk_hv_percent=10.1, 
                                        vk_mv_percent=10.1, vk_lv_percent=10.1, vkr_hv_percent=0.266667, 
                                        vkr_mv_percent=0.033333, vkr_lv_percent=0.04, pfe_kw=0, i0_percent=0, 
                                        shift_mv_degree=30, shift_lv_degree=30, tap_side="hv", tap_neutral=0, tap_min=-8, 
                                        tap_max=8, tap_step_percent=1.25, tap_pos=0, name='HV-MV-MV-Trafo', 
                                        tap_changer_type="Ratio")

# show transformer3w table
net.trafo3w

,name,std_type,hv_bus,mv_bus,lv_bus,sn_hv_mva,sn_mv_mva,sn_lv_mva,vn_hv_kv,vn_mv_kv,...,id_characteristic_table,tap_dependency_table,in_service,vk0_hv_percent,vk0_mv_percent,vk0_lv_percent,vkr0_hv_percent,vkr0_mv_percent,vkr0_lv_percent,vector_group
0,HV-MV-MV-Trafo,None,33,36,37,40.0,15.0,25.0,110.0,20.0,...,<NA>,False,True,NaN,NaN,NaN,NaN,NaN,NaN,None


### Switches

In [19]:
# Bus-line switches
mv_buses = net.bus[(net.bus.vn_kv == 10) | (net.bus.vn_kv == 20)].index
mv_ls = net.line[(net.line.from_bus.isin(mv_buses)) & (net.line.to_bus.isin(mv_buses))]
for _, line in mv_ls.iterrows():
        create_switch(net, line.from_bus, line.name, et='l', closed=True, type='LBS', name=f"Switch {net.bus.name.at[line.from_bus]} - {line['name']}")
        create_switch(net, line.to_bus, line.name, et='l', closed=True, type='LBS', name=f"Switch {net.bus.name.at[line.to_bus]} - {line['name']}")

# open switch
open_switch_id = net.switch[(net.switch.name == 'Switch Bus MV5 - MV Line5')].index
net.switch.closed.loc[open_switch_id] = False

#show only medium voltage switch table
net.switch[net.switch.bus.isin(mv_buses)]

,bus,element,et,type,closed,name,z_ohm,in_ka
48,37,6,l,LBS,True,Switch Bus MV0 - MV Line1,0.0,NaN
49,38,6,l,LBS,True,Switch Bus MV1 - MV Line1,0.0,NaN
50,38,7,l,LBS,True,Switch Bus MV1 - MV Line2,0.0,NaN
51,39,7,l,LBS,True,Switch Bus MV2 - MV Line2,0.0,NaN
52,39,8,l,LBS,True,Switch Bus MV2 - MV Line3,0.0,NaN
53,40,8,l,LBS,True,Switch Bus MV3 - MV Line3,0.0,NaN
54,40,9,l,LBS,True,Switch Bus MV3 - MV Line4,0.0,NaN
55,41,9,l,LBS,True,Switch Bus MV4 - MV Line4,0.0,NaN
56,41,10,l,LBS,True,Switch Bus MV4 - MV Line5,0.0,NaN
57,42,10,l,LBS,False,Switch Bus MV5 - MV Line5,0.0,NaN


### Loads

<img src="pics/example_network_loads_mv.png"  width="50%">

In [20]:
mv_loads = pd.read_csv('example_advanced/mv_loads.csv', sep=';', header=0, decimal=',')
for _, load in mv_loads.iterrows():
    bus_idx = get_element_index(net, "bus", load.bus)
    create_load(net, bus_idx, p_mw=load.p, q_mvar=load.q, name=load.load_name)

# show only medium voltage loads
net.load[net.load.bus.isin(mv_buses)]

,name,bus,p_mw,q_mvar,const_z_p_percent,const_i_p_percent,const_z_q_percent,const_i_q_percent,sn_mva,scaling,in_service,type
5,Further MV-Rings,37,6.0,2.00,0.0,0.0,0.0,0.0,NaN,1.0,True,wye
6,Industry Load,36,18.0,4.00,0.0,0.0,0.0,0.0,NaN,1.0,True,wye
7,LV Net 1,38,0.4,0.10,0.0,0.0,0.0,0.0,NaN,1.0,True,wye
8,LV Net 2,39,0.4,0.06,0.0,0.0,0.0,0.0,NaN,1.0,True,wye
9,LV Net 3,40,0.4,0.06,0.0,0.0,0.0,0.0,NaN,1.0,True,wye
10,LV Net 5,42,0.4,0.06,0.0,0.0,0.0,0.0,NaN,1.0,True,wye
11,LV Net 6,43,0.4,0.06,0.0,0.0,0.0,0.0,NaN,1.0,True,wye
12,LV Net 7,44,0.4,0.06,0.0,0.0,0.0,0.0,NaN,1.0,True,wye


### Static generators

<img src="pics/example_network_sgens_mv.png"  width="50%">

In [21]:
mv_sgens = pd.read_csv('example_advanced/mv_sgens.csv', sep=';', header=0, decimal=',')
for _, sgen in mv_sgens.iterrows():
    bus_idx = get_element_index(net, "bus", sgen.bus)
    create_sgen(net, bus_idx, p_mw=sgen.p, q_mvar=sgen.q, sn_mva=sgen.sn, type=sgen.type, name=sgen.sgen_name)

# show only medium voltage static generators
net.sgen[net.sgen.bus.isin(mv_buses)]

,name,bus,p_mw,q_mvar,min_q_mvar,max_q_mvar,sn_mva,scaling,controllable,id_q_capability_characteristic,reactive_capability_curve,curve_style,in_service,type,current_source
1,Biogas plant,43,0.5,0.00,NaN,NaN,750.0,1.0,False,<NA>,False,NaN,True,SGEN,True
2,Further MV Generator,37,0.5,0.05,NaN,NaN,1000.0,1.0,False,<NA>,False,NaN,True,SGEN,True
3,Industry Generator,36,15.0,3.00,NaN,NaN,20000.0,1.0,False,<NA>,False,NaN,True,SGEN,True
4,PV Park,42,2.0,0.10,NaN,NaN,5000.0,1.0,False,<NA>,False,NaN,True,PV,True


## Low voltage level

### Busses

<img src="pics/example_network_buses_lv.png" width="50%">

In [22]:
create_bus(net, name='Bus LV0', vn_kv=0.4, type='n')
for i in range(1, 6):
    create_bus(net, name='Bus LV1.%s' % i, vn_kv=0.4, type='m')
for i in range(1, 5):
    create_bus(net, name='Bus LV2.%s' % i, vn_kv=0.4, type='m')
create_bus(net, name='Bus LV2.2.1', vn_kv=0.4, type='m')
create_bus(net, name='Bus LV2.2.2', vn_kv=0.4, type='m')

# show only low voltage buses
lv_buses = net.bus[net.bus.vn_kv == 0.4]
lv_buses

,name,vn_kv,type,zone,in_service,geo
45,Bus LV0,0.4,n,None,True,None
46,Bus LV1.1,0.4,m,None,True,None
47,Bus LV1.2,0.4,m,None,True,None
48,Bus LV1.3,0.4,m,None,True,None
49,Bus LV1.4,0.4,m,None,True,None
50,Bus LV1.5,0.4,m,None,True,None
51,Bus LV2.1,0.4,m,None,True,None
52,Bus LV2.2,0.4,m,None,True,None
53,Bus LV2.3,0.4,m,None,True,None
54,Bus LV2.4,0.4,m,None,True,None


### Lines

<img src="pics/example_network_lines_lv.png" width="50%">

In [23]:
# create lines
lv_lines = pd.read_csv('example_advanced/lv_lines.csv', sep=';', header=0, decimal=',')
for _, lv_line in lv_lines.iterrows():
    from_bus = get_element_index(net, "bus", lv_line.from_bus)
    to_bus = get_element_index(net, "bus", lv_line.to_bus)
    create_line(net, from_bus, to_bus, length_km=lv_line.length, std_type=lv_line.std_type, name=lv_line.line_name)

# show only low voltage lines
net.line[net.line.from_bus.isin(lv_buses.index)]

,name,std_type,from_bus,to_bus,length_km,r_ohm_per_km,x_ohm_per_km,c_nf_per_km,g_us_per_km,max_i_ka,df,parallel,type,in_service,geo
14,LV Line1.1,NAYY 4x120 SE,45,46,0.08,0.2250,0.08,264.0,0.0,0.242,1.0,1,cs,True,None
15,LV Line1.2,NAYY 4x120 SE,46,47,0.08,0.2250,0.08,264.0,0.0,0.242,1.0,1,cs,True,None
16,LV Line1.3,NAYY 4x120 SE,47,48,0.08,0.2250,0.08,264.0,0.0,0.242,1.0,1,cs,True,None
17,LV Line1.4,NAYY 4x120 SE,48,49,0.08,0.2250,0.08,264.0,0.0,0.242,1.0,1,cs,True,None
18,LV Line1.6,NAYY 4x120 SE,49,50,0.08,0.2250,0.08,264.0,0.0,0.242,1.0,1,cs,True,None
19,LV Line2.1,NAYY 4x120 SE,45,51,0.12,0.2250,0.08,264.0,0.0,0.242,1.0,1,cs,True,None
20,LV Line2.2,NAYY 4x120 SE,51,52,0.12,0.2250,0.08,264.0,0.0,0.242,1.0,1,cs,True,None
21,LV Line2.3,15-AL1/3-ST1A 0.4,52,53,0.12,1.8769,0.35,11.0,0.0,0.105,1.0,1,ol,True,None
22,LV Line2.4,15-AL1/3-ST1A 0.4,53,54,0.12,1.8769,0.35,11.0,0.0,0.105,1.0,1,ol,True,None
23,LV Line2.2.1,15-AL1/3-ST1A 0.4,52,55,0.12,1.8769,0.35,11.0,0.0,0.105,1.0,1,ol,True,None


### Transformer

<img src="pics/example_network_trafos_lv.png" width="50%">

In [24]:
hv_bus = get_element_index(net, "bus", "Bus MV4")
lv_bus = get_element_index(net, "bus","Bus LV0")
create_transformer_from_parameters(net, hv_bus, lv_bus, sn_mva=.4, vn_hv_kv=10, vn_lv_kv=0.4, vkr_percent=1.325, vk_percent=4, pfe_kw=0.95, i0_percent=0.2375, tap_side="hv", tap_neutral=0, tap_min=-2, tap_max=2, tap_step_percent=2.5, tap_changer_type="Ratio", tp_pos=0, shift_degree=150, name='MV-LV-Trafo')

#show only low voltage transformer
net.trafo[net.trafo.lv_bus.isin(lv_buses.index)]

,name,std_type,hv_bus,lv_bus,sn_mva,vn_hv_kv,vn_lv_kv,vk_percent,vkr_percent,pfe_kw,...,tap_step_degree,tap_pos,tap_changer_type,id_characteristic_table,tap_dependency_table,parallel,df,in_service,tp_pos,oltc
1,MV-LV-Trafo,None,41,45,0.4,10.0,0.4,4.0,1.325,0.95,...,NaN,0.0,Ratio,<NA>,False,1,1.0,True,0.0,False


### Switches

In [25]:
lv_buses
# Bus-line switches
lv_ls = net.line[(net.line.from_bus.isin(lv_buses.index)) & (net.line.to_bus.isin(lv_buses.index))]
for _, line in lv_ls.iterrows():
        create_switch(net, line.from_bus, line.name, et='l', closed=True, type='LBS', name=f"Switch {net.bus.name.at[line.from_bus]} - {line['name']}")
        create_switch(net, line.to_bus, line.name, et='l', closed=True, type='LBS', name=f"Switch {net.bus.name.at[line.to_bus]} - {line['name']}")

# Trafo-line switches
create_switch(net, get_element_index(net, "bus", 'Bus MV4'), get_element_index(net, "trafo", 'MV-LV-Trafo'), et='t', closed=True, type='LBS', name='Switch MV4 - MV-LV-Trafo')
create_switch(net, get_element_index(net, "bus", 'Bus LV0'), get_element_index(net, "trafo", 'MV-LV-Trafo'), et='t', closed=True, type='LBS', name='Switch LV0 - MV-LV-Trafo')

# show only low vvoltage switches
net.switch[net.switch.bus.isin(lv_buses.index)]


,bus,element,et,type,closed,name,z_ohm,in_ka
64,45,14,l,LBS,True,Switch Bus LV0 - LV Line1.1,0.0,NaN
65,46,14,l,LBS,True,Switch Bus LV1.1 - LV Line1.1,0.0,NaN
66,46,15,l,LBS,True,Switch Bus LV1.1 - LV Line1.2,0.0,NaN
67,47,15,l,LBS,True,Switch Bus LV1.2 - LV Line1.2,0.0,NaN
68,47,16,l,LBS,True,Switch Bus LV1.2 - LV Line1.3,0.0,NaN
69,48,16,l,LBS,True,Switch Bus LV1.3 - LV Line1.3,0.0,NaN
70,48,17,l,LBS,True,Switch Bus LV1.3 - LV Line1.4,0.0,NaN
71,49,17,l,LBS,True,Switch Bus LV1.4 - LV Line1.4,0.0,NaN
72,49,18,l,LBS,True,Switch Bus LV1.4 - LV Line1.6,0.0,NaN
73,50,18,l,LBS,True,Switch Bus LV1.5 - LV Line1.6,0.0,NaN


### Loads

<img src="pics/example_network_loads_lv.png" width="50%">

In [26]:
lv_loads = pd.read_csv('example_advanced/lv_loads.csv', sep=';', header=0, decimal=',')
for _, load in lv_loads.iterrows():
    bus_idx = get_element_index(net, "bus", load.bus)
    create_load(net, bus_idx, p_mw=load.p, q_mvar=load.q, name=load.load_name)
    
# show only low voltage loads
net.load[net.load.bus.isin(lv_buses.index)]

,name,bus,p_mw,q_mvar,const_z_p_percent,const_i_p_percent,const_z_q_percent,const_i_q_percent,sn_mva,scaling,in_service,type
13,Further LV-Feeders Load,45,0.10,0.10,0.0,0.0,0.0,0.0,NaN,1.0,True,wye
14,Residential Load,46,0.01,0.03,0.0,0.0,0.0,0.0,NaN,1.0,True,wye
15,Residential Load(1),47,0.01,0.03,0.0,0.0,0.0,0.0,NaN,1.0,True,wye
16,Residential Load(2),48,0.01,0.03,0.0,0.0,0.0,0.0,NaN,1.0,True,wye
17,Residential Load(3),49,0.01,0.03,0.0,0.0,0.0,0.0,NaN,1.0,True,wye
18,Residential Load(4),50,0.01,0.03,0.0,0.0,0.0,0.0,NaN,1.0,True,wye
19,Rural Load,51,0.01,0.03,0.0,0.0,0.0,0.0,NaN,1.0,True,wye
20,Rural Load(1),52,0.01,0.03,0.0,0.0,0.0,0.0,NaN,1.0,True,wye
21,Rural Load(2),53,0.01,0.03,0.0,0.0,0.0,0.0,NaN,1.0,True,wye
22,Rural Load(3),54,0.01,0.03,0.0,0.0,0.0,0.0,NaN,1.0,True,wye


### Static generators

<img src="pics/example_network_sgens_lv.png"  width="50%">

In [27]:
lv_sgens = pd.read_csv('example_advanced/lv_sgens.csv', sep=';', header=0, decimal=',')
for _, sgen in lv_sgens.iterrows():
    bus_idx = get_element_index(net, "bus", sgen.bus)
    create_sgen(net, bus_idx, p_mw=sgen.p, q_mvar=sgen.q, sn_mva=sgen.sn, type=sgen.type, name=sgen.sgen_name)

# show only low voltage static generators
net.sgen[net.sgen.bus.isin(lv_buses.index)]

,name,bus,p_mw,q_mvar,min_q_mvar,max_q_mvar,sn_mva,scaling,controllable,id_q_capability_characteristic,reactive_capability_curve,curve_style,in_service,type,current_source
5,PV,46,0.06,0.0,NaN,NaN,0.012,1.0,False,<NA>,False,NaN,True,PV,True
6,PV(1),48,0.05,0.0,NaN,NaN,0.010,1.0,False,<NA>,False,NaN,True,PV,True
7,PV(2),53,0.05,0.0,NaN,NaN,0.010,1.0,False,<NA>,False,NaN,True,PV,True
8,PV(3),54,0.05,0.0,NaN,NaN,0.010,1.0,False,<NA>,False,NaN,True,PV,True
9,PV(4),55,0.05,0.0,NaN,NaN,0.010,1.0,False,<NA>,False,NaN,True,PV,True
10,PV(5),56,0.05,0.0,NaN,NaN,0.010,1.0,False,<NA>,False,NaN,True,PV,True


## Run a Power Flow

In [28]:
runpp(net, calculate_voltage_angles=True, init="dc")
net

This pandapower network includes the following parameter tables:
   - bus (57 elements)
   - load (25 elements)
   - sgen (11 elements)
   - gen (1 element)
   - switch (88 elements)
   - shunt (1 element)
   - ext_grid (1 element)
   - line (25 elements)
   - trafo (2 elements)
   - trafo3w (1 element)
   - impedance (1 element)
   - xward (2 elements)
 and the following results tables:
   - res_bus (57 elements)
   - res_line (25 elements)
   - res_trafo (2 elements)
   - res_trafo3w (1 element)
   - res_impedance (1 element)
   - res_ext_grid (1 element)
   - res_load (25 elements)
   - res_sgen (11 elements)
   - res_shunt (1 element)
   - res_gen (1 element)
   - res_xward (2 elements)
   - res_switch (88 elements)